In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [42]:
spark = SparkSession.builder \
  .appName('facts_table') \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()  
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [3]:
table = "becade_rgarciaf.stg_compras"
stg_compras = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()

In [4]:
table_3 = "becade_rgarciaf.stg_external_procuts"
stg_external_products = spark.read \
  .format("bigquery") \
  .option("table", table_3) \
  .load()


In [5]:
from pyspark.sql.functions import year, month, dayofmonth,col
from pyspark.sql.functions import sum, col, desc, avg, count

In [6]:
compras_2 = stg_compras.select("compra_id","client_id","product_id","cantidad","precio","isprime","metodo_pago",
    year("fecha_compra").alias('year'), 
    month("fecha_compra").alias('month'), 
    dayofmonth("fecha_compra").alias('day')
)

In [9]:
compras_2.show(10)

+-------------------+-----------------+----------+--------+------+-------+-----------+----+-----+---+
|          compra_id|        client_id|product_id|cantidad|precio|isprime|metodo_pago|year|month|day|
+-------------------+-----------------+----------+--------+------+-------+-----------+----+-----+---+
|831-0776824-5815830|831-175061-77-427|B00N69D6AS|       1|236.99|   true|  **** 0012|2011|   12| 10|
|831-2232014-2295491|831-175061-77-427|B00N69D6AS|       1|236.99|   true|  **** 0012|2011|   12| 30|
|831-7168743-3850172|831-175061-77-427|B00N69D6AS|       1|236.99|   true|  **** 0012|2012|   10| 27|
|831-6524226-3868173|831-175061-77-427|B00N69D6AS|       1|236.99|   true|  **** 0012|2012|   11|  1|
|831-7422339-0701440|831-175061-77-427|B00N69D6AS|       1|236.99|   true|  **** 0012|2012|   11|  9|
|831-0474586-3215945|831-175061-77-427|B00N69D6AS|       1|236.99|   true|  **** 0012|2012|   11| 17|
|831-1807579-4683398|831-175061-77-427|B00N69D6AS|       1|236.99|   true|  **** 0

In [10]:
from pyspark.sql.functions import countDistinct

In [74]:
compras_anuales = compras_2.groupBy("year","product_id")\
    .agg(sum("cantidad").alias("cantidad_de_unidades"),countDistinct("client_id").alias("numero_de_clientes"),countDistinct("compra_id").alias("cantidad_de_ventas")) 

In [67]:
compras_anuales.orderBy("product_id","year").show(11)

+----+----------+--------------------+------------------+------------------+
|year|product_id|cantidad_de_unidades|numero_de_clientes|cantidad_de_ventas|
+----+----------+--------------------+------------------+------------------+
|2010|B00N69D6AS|                1927|               100|              1800|
|2011|B00N69D6AS|                1848|               100|              1751|
|2012|B00N69D6AS|                1881|               100|              1761|
|2013|B00N69D6AS|                1852|               100|              1723|
|2014|B00N69D6AS|                1926|               100|              1812|
|2015|B00N69D6AS|                1913|               100|              1784|
|2016|B00N69D6AS|                1893|               100|              1773|
|2017|B00N69D6AS|                1858|               100|              1731|
|2018|B00N69D6AS|                1955|               100|              1809|
|2019|B00N69D6AS|                1939|               100|              1828|

In [75]:
products_fact_table = compras_anuales.join(stg_external_products,stg_external_products.product_id ==  compras_anuales.product_id,"inner").select(compras_anuales.year, compras_anuales.product_id,compras_anuales.cantidad_de_unidades,compras_anuales.numero_de_clientes,compras_anuales.cantidad_de_ventas,stg_external_products.evaluate_rate)

In [76]:
products_fact_table.orderBy("product_id","year").show(21)

+----+----------+--------------------+------------------+------------------+------------------+
|year|product_id|cantidad_de_unidades|numero_de_clientes|cantidad_de_ventas|     evaluate_rate|
+----+----------+--------------------+------------------+------------------+------------------+
|2010|B00N69D6AS|                1927|               100|              1800|4.4 out of 5 stars|
|2011|B00N69D6AS|                1848|               100|              1751|4.4 out of 5 stars|
|2012|B00N69D6AS|                1881|               100|              1761|4.4 out of 5 stars|
|2013|B00N69D6AS|                1852|               100|              1723|4.4 out of 5 stars|
|2014|B00N69D6AS|                1926|               100|              1812|4.4 out of 5 stars|
|2015|B00N69D6AS|                1913|               100|              1784|4.4 out of 5 stars|
|2016|B00N69D6AS|                1893|               100|              1773|4.4 out of 5 stars|
|2017|B00N69D6AS|                1858|  

In [77]:
products_fact_table.count()

1727

In [78]:
products_fact_table.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.products_fact_table") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()